# Revisão para a P2

<h1>Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#(Aula-22)-Identificação-e-Assinatura-de-Schnorr" data-toc-modified-id="(Aula-22)-Identificação-e-Assinatura-de-Schnorr-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>(Aula 22) Identificação e Assinatura de Schnorr</a></span><ul class="toc-item"><li><span><a href="#O-esquema-de-Identificação" data-toc-modified-id="O-esquema-de-Identificação-1.1"><span class="toc-item-num">1.1&nbsp;&nbsp;</span>O esquema de Identificação</a></span></li><li><span><a href="#Quem-não-conhece-$x$-não-consegue-impersonar-Alice" data-toc-modified-id="Quem-não-conhece-$x$-não-consegue-impersonar-Alice-1.2"><span class="toc-item-num">1.2&nbsp;&nbsp;</span>Quem não conhece $x$ não consegue impersonar Alice</a></span></li><li><span><a href="#O-protocolo-é-uma-prova-de-Conhecimento-Zero" data-toc-modified-id="O-protocolo-é-uma-prova-de-Conhecimento-Zero-1.3"><span class="toc-item-num">1.3&nbsp;&nbsp;</span>O protocolo é uma prova de Conhecimento-Zero</a></span></li><li><span><a href="#Assinatura-de-Schnorr-usando-transformação-de-Fiat-Shamir" data-toc-modified-id="Assinatura-de-Schnorr-usando-transformação-de-Fiat-Shamir-1.4"><span class="toc-item-num">1.4&nbsp;&nbsp;</span>Assinatura de Schnorr usando transformação de Fiat-Shamir</a></span></li></ul></li><li><span><a href="#(Aula-20)-Oblivious-Transfer" data-toc-modified-id="(Aula-20)-Oblivious-Transfer-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>(Aula 20) Oblivious Transfer</a></span><ul class="toc-item"><li><span><a href="#Cara-ou-coroa-pelo-telefone" data-toc-modified-id="Cara-ou-coroa-pelo-telefone-2.1"><span class="toc-item-num">2.1&nbsp;&nbsp;</span>Cara-ou-coroa pelo telefone</a></span></li><li><span><a href="#Oblivious-Transfer:-caso-geral" data-toc-modified-id="Oblivious-Transfer:-caso-geral-2.2"><span class="toc-item-num">2.2&nbsp;&nbsp;</span>Oblivious Transfer: caso geral</a></span></li></ul></li><li><span><a href="#(Aula-23)-Hashes" data-toc-modified-id="(Aula-23)-Hashes-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>(Aula 23) Hashes</a></span></li></ul></div>

## (Aula 22) Identificação e Assinatura de Schnorr 

### O esquema de Identificação

Seja $G$ um grupo multiplicativo de ordem $q$ e gerador $g$. 
Considere $x$ a chave secreta de Alice e $y = g^x$ a sua chave pública.

O esquema de identificação de Schnorr é dado pelo protocolo a seguir.

$$
\begin{array}{@{}l@{}c@{}l@{}}
\text{Alice} & \qquad \qquad \qquad & \text{Bob} \\
s \gets Z_q && \\
I = g^s & \xrightarrow{\textstyle I} & \\
& \xleftarrow{\textstyle c} & c \gets Z_q \\
r = cx + s & \xrightarrow{\textstyle r} & 
\begin{cases}
    \text{Se } g^ry^{-c} = I, \textbf{aceite} \\
    \text{Senão } \textbf{rejeite}
\end{cases}
\end{array}
$$

Se a Alice está falando a verdade, então $g^r y^{-c} = g^{cx + s}g^{-cx} = g^s = I$.

Qual a probabilidade de um atacante que conhece $y$ conseguir impersonar Alice?

A demonstração só funciona por conta da ordem em que as mensagens são enviadas.
Se eu quero garantir que I foi gerado antes de c, eu posso fazer c = HASH(I).

### Quem não conhece $x$ não consegue impersonar Alice

**Suponha** que um adversário consegue impersonar Alice com probabilidade não desprezável, isto é:
$$
\Pr(\text{Impersonar Alice}) > 1/q.
$$

Então, para um mesmo valor de $I$, o adversário consegue dar respostas válidas para ao menos **dois desafios diferentes** $c_1$ e $c_2$. Isso é facilmente visto poir se o adversário conseguisse respoder apenas um $c$, então a probabilidade de ele conseguir atacar seria $\Pr(\text{Impersonar Alice}) = 1/q$.

Sejam $r_1$ e $r_2$ as respostas para $c_1$ e $c_2$, respectivamente.
Se ele consegue responder dois desafios para um mesmo $I$, então devemos ter que 
$$
\begin{align}
    g^{r_1}y^{-c_1} &= g^{r_2}y^{-c_2} = I\\
    \implies g^\tfrac{r_1 - r_2}{c_1-c_2} &= y
\end{align}
$$
Ou seja, o atacante sabe uma solução para o log discreto para $y = g^x$.

### O protocolo é uma prova de Conhecimento-Zero
Sobre Zero-Knowledge, ver tutorial do Oded Goldreich
http://www.wisdom.weizmann.ac.il/~oded/zk-tut02.html \
Sobre protocolos, há um texto do Ivan Damgard: https://users-cs.au.dk/~ivan/Sigma.pdf

Veremos que, **se o verificador é honesto**, este protocolo é uma prova de conhecimento-zero.\
Não se sabe se o mesmo vale para verificadores desonestos.

Este fato pode ser comprovado se perguntando o que exatamente o verificador (Bob) aprende após 
interagir com o provador (Alice):
1. A distribuição de $c$ é uniforme em $Z_q$
2. A distribuição de $r$ é uniforme em $Z_q$ pois $s$ foi escolhido aleatoriamente de forma uniforme em $Z_q$
3. $I = g^ry^{-c}$ pode ser calculado por Bob

Portanto, Bob não aprende nada!

**Se Bob quisesse, poderia brincar desse jogo sozinho.**


Mas note que o protocolo é totalmente inseguro num ataque de ataque de pessoa-no-meio! \
Por que então ele é interessante?

### Assinatura de Schnorr usando transformação de Fiat-Shamir

Ideia básica:
1. Provador roda o protocolo de identificação sozinho
2. Usa como desafio $c$ o Hash de $(m, I)$, e portanto não tem controle suficiente sobre o desafio.

O protocolo abaixo acontece na cabeça de Alice:
$$
\begin{array}{@{}l@{}c@{}l@{}}
\text{Alice} & \qquad \qquad \qquad & \text{Amigo imaginário da Alice} \\
s \gets Z_q && \\
I = g^s & \xrightarrow{\textstyle I} & \\
& \xleftarrow{\textstyle c} & c = \text{hash}(I, m)\\
r = cx + s && \\
\text{Devolve assinatura: } (c, r) &&
\end{array}
$$

**Verificação da assinatura.**
1. Seja $I_v = g^ry^{-c}$
2. Aceite se $\text{hash}(I_v, m) = c$, 

## (Aula 20) Oblivious Transfer
Cara-ou-coroa pelo telefone: https://dl.acm.org/doi/pdf/10.1145/1008908.1008911 \
Obs: Oblivious transfer é a coisa mais maluca que a gente verá hoje.

### Cara-ou-coroa pelo telefone

Seja $G$ um grupo de ordem $q$ e gerador $g$. 
Considere $x$ a chave pública de Alice e $y = g^x$ a sua chave privada.

O esquema de identificação de Schnorr é dado pelo protocolo a seguir.

$$
\begin{array}{@{}l@{}c@{}l@{}}
\text{Alice} & \qquad \qquad \qquad & \text{Bob} \\
p, q \gets^$ \text{Primos grandes} && \\
n = pq & \xrightarrow{\textstyle n} & \text{Se $n$ for potência de primo, ou par, Alice perde} \\ 
&& x \gets^$ \{1, \ldots, n - 1\}\\ 
& \xleftarrow{\textstyle a} & a = x^2 \mod n\\
b_1, b_2, b_3, b_4 = \text{Quatro raízes quadradas de $a$} && \\
b \gets^$ \{b_1, b_2, b_3, b_4\} & \xrightarrow{\textstyle b} &
\begin{cases}
    \text{Se } b = x \text{ ou } b = n - x,  \textbf{Alice ganha} \\
    \text{Senão } \textbf{Bob ganha e deve provar que ganha} 
\end{cases} \\
&& x^2 - b^2 = 0 \mod n \implies (x + b)(x - b) = kn \\
& \xleftarrow{\textstyle r} & r = \gcd(x + b, n) \\
\begin{cases}
    \text{Se } r = p \text{ ou } r = q,  \textbf{Alice aceita Bob ganhou} \\
    \text{Senão } \textbf{Alice aceita que ganhou} 
\end{cases} \\
\end{array}
$$


Antes da última mensagem $r$ ser enviada, este é um caso particular de Oblivious Transfer:
1. Quando Alice envia $b$ para Bob, ela não sabe se mandou um bit $0$ ou se mandou um bit $1$.
2. Além disso, ninguém sniffando a rede sabe qual bit Bob recebeu!

Imaginem se em vez de bits, fossem mensagem encriptadas.
Seria possível você comprar um livro digital na Amazon de forma que:
1. A a Amazon não soubesse qual livro você comprou.
2. Você não tivesse informação nenhuma sobre os livros que você não comprou.
3. Ninguém siniffando a rede soubesse que livro você comprou.

### Oblivious Transfer: caso geral
Ver https://eprint.iacr.org/2015/267.pdf

Seja $p$ um primo imenso e $g$ um gerador de $Z_p$. \
Suponha que bob gostaria de baixar um documento $M_c$ de Alice, de forma privada (até mesmo de Alice).

O esquema SIOT (Simplest Oblivious Transfer Protocol) permite que isso ocorra da seguinte forma.

$$
\begin{array}{@{}l@{}c@{}l@{}}
\text{Alice} & \qquad \qquad \qquad & \text{Bob} \\
a \gets^$ Z_p && \\
A = g^a & \xrightarrow{\textstyle I} & b \gets^$ Z_p && \\
&& 
\begin{cases}
    \text{Se } c = 0, \text{ então } B = g^b \\
    \text{Se } c = 1, \text{ então } B = Ag^b 
\end{cases} \\
& \xleftarrow{\textstyle B} & \\
k_0 = B^a && \\
k_1 = (B/A)^a && \\
e_0 = E_{k_0}(M_0) && \\
e_1 = E_{k_1}(M_1) & \xrightarrow{\textstyle e_0, e_1} & k_R = A^b \\
&& M_c = D_{k_R}(e_c)
\end{array}
$$

Essencialmente o que está acontecendo é uma troca de chaves de Diffie-Hellman especial em que:
1. Bob escolhe um documento de interesse, $M_0$ ou $M_1$
2. Alice encripta os dois documentos um com a chave $k_0$, outro com a chave $k_1$.
3. Bob só consegue decriptar um dos documentos, relativo à sua escolha anterior.
4. Alice não sabe qual documento Bob consegue decriptar.

**Parte mais difícil de ver.** Por que Bob não consegue computar a chave que ele não escolheu?

Suponha que bob escolheu $c = 0$. \
Então temos que
1. $B = g^b$.
2. $k_0 = B^a = g^{ab}$ 
3. $k_1 = (B/A)^a = g^{(b - a)a} = g^{ab - a^2} = \frac{k_R}{g^{a^2}}$

Como memso conhecendo $A = g^a$ é difícil saber $a^2$, então Bob não consegue descobrir $k_1$.

## (Aula 23) Hashes
Como já vimos as propriedades de hashes há algum tempo, hoje vamos brincar de hacker fazendo alguns exercícios com o hashcat.

In [2]:
import hashlib

In [4]:
h = hashlib.sha256('xeste123'.encode('utf-8')).hexdigest()
f = open('test.hash', 'w')
f.write(h)
f.close()